In [2]:
!pip install transformers torch

In [3]:
import pandas as pd
import json

# Данные

In [6]:
m = ["/content/sample_data/Cosmos Saint -Petersburg Pulkovskaya Hotel.txt", "/content/sample_data/Cosmos St.Petersburg Olympia Garden Hotel.txt", "/content/sample_data/Crown Hotel St. Petersburg.txt", "/content/sample_data/VALO Ramada Plaza by Wyndham St.Petersburg.txt", "/content/sample_data/Апарт-отель Artstudio Moskovsky.txt", "/content/sample_data/Апарт-отель Avenue-Apart на Малом.txt", "/content/sample_data/Отель Nevskiy Eclectic by AKYAN.txt", "/content/sample_data/Отель Palace Bridge Hotel.txt", "/content/sample_data/Отель Renome.txt", "/content/sample_data/Отель Vasilievsky.txt", "/content/sample_data/Отель Космос Прибалтийская.txt", "/content/sample_data/Отель Нептун.txt", "/content/sample_data/Отель Номера на Невском 111.txt", "/content/sample_data/Отель Номера на Садовой.txt", "/content/sample_data/Отель Полюстрово.txt", "/content/sample_data/Отель Станция Премьер V18.txt", "/content/sample_data/Хостел Guten Duck St. Petersburg.txt", "/content/sample_data/Хостел Railway Capsules.txt"]
data_list = []
for i in m:
  file = open(i, encoding='utf-8')
  ot = file.readlines()
  for i in ot:
    n = i[:-1].split(' | ')
    if '.' in n[1]:
      n[1] = n[1][0]
    n[1] = int(n[1])
    data_list.append({
            'Rating': n[1],
            'Text': n[0],
           })

df = pd.DataFrame(data_list)
df

,Rating,Text
0,7,"Приветливый персонал, номер соответствует ожид..."
1,9,Прекрасный видОтсутствие мини шампуня и бальза...
2,8,"Чисто, спокойно. Удобно добираться до Пулково...."
3,9,остановка 39 автобуса рядом до аэропорта Пулко...
4,8,"Все понравилось! Рекомендую! Метро рядом, 10 м..."
...,...,...
7890,10,
7891,1,"МестоположениеРаньше был не дорогой хостел, с ..."
7892,4,"Уборщица постоянно убирает, моет полы, туалеты..."
7893,7,"сервис мог бы быть лучше, хотя в целом - капсу..."


# Подготовка текста

In [7]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Загрузка стоп-слов и инициализация лемматизатора
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('russian'))
lemmatizer = WordNetLemmatizer()

# Функция для предобработки текста
def preprocess_text(text):
    # Приведение текста к нижнему регистру
    text = text.lower()

    # Удаление ссылок и спецсимволов
    text = re.sub(r"http\S+|www\S+|https\S+", '', text, flags=re.MULTILINE)
    text = re.sub(r'\@\w+|\#','', text)

    # Удаление чисел и пунктуации
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)

    # Токенизация текста
    words = word_tokenize(text)

    # Удаление стоп-слов и лемматизация
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]

    return " ".join(words)

# Пример использования
df['cleaned_review'] = df['Text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Инициализация TF-IDF векторайзера
tfidf = TfidfVectorizer(max_features=5000)

# Преобразование обучающих данных
X = tfidf.fit_transform(df['cleaned_review'])
df

,Rating,Text,cleaned_review
0,7,"Приветливый персонал, номер соответствует ожид...",приветливый персонал номер соответствует ожида...
1,9,Прекрасный видОтсутствие мини шампуня и бальза...,прекрасный видотсутствие мини шампуня бальзама...
2,8,"Чисто, спокойно. Удобно добираться до Пулково....",чисто спокойно удобно добираться пулково хорош...
3,9,остановка 39 автобуса рядом до аэропорта Пулко...,остановка автобуса рядом аэропорта пулково оте...
4,8,"Все понравилось! Рекомендую! Метро рядом, 10 м...",понравилось рекомендую метро рядом минут остан...
...,...,...,...
7890,10,,
7891,1,"МестоположениеРаньше был не дорогой хостел, с ...",местоположениераньше дорогой хостел нормальным...
7892,4,"Уборщица постоянно убирает, моет полы, туалеты...",уборщица постоянно убирает моет полы туалеты с...
7893,7,"сервис мог бы быть лучше, хотя в целом - капсу...",сервис мог хотя целом капсульный отель центре ...


# Обучение модели

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

In [10]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [11]:
reviews = df['cleaned_review'].to_list()
ratings = df['Rating'].to_list()

# Разделение данных на тренировочную и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(reviews, ratings, test_size=0.4, random_state=0)

# Преобразование текста в TF-IDF
vectorizer = TfidfVectorizer(max_features=5000)
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Модель классификации
model = LogisticRegression()
model.fit(X_train_tfidf, y_train)

# Предсказания и оценка
y_pred = model.predict(X_test_tfidf)
print(f'Accuracy: {accuracy_score(y_test, y_pred)}')
print(classification_report(y_test, y_pred))

Accuracy: 0.9053198226725776
              precision    recall  f1-score   support

           1       0.95      0.67      0.78        27
           4       0.88      0.70      0.78        20
           5       0.85      0.74      0.79        46
           6       0.92      0.70      0.79        82
           7       0.90      0.82      0.86       346
           8       0.93      0.88      0.91       562
           9       0.91      0.87      0.89       561
          10       0.90      0.97      0.93      1514

    accuracy                           0.91      3158
   macro avg       0.90      0.79      0.84      3158
weighted avg       0.91      0.91      0.90      3158

